In [1]:
import os
import sys
import scipy.io
import numpy as np
import tensorflow as tf
import time
import scipy.sparse
import scipy.sparse.linalg
from random import shuffle
norms = [66.71730, 33.34396, 16.92447, 372.3057, 186.1538]

In [2]:
t1 = time.time()
mat = scipy.io.loadmat('/Users/aditya/LocalDocs/XRAYDATA/Walnut/Data82.mat')
norm = 0
mult = 1
c1 = 82*mult
c2 = 9840*mult

Af = mat["A"]/norms[norm] #norm
m = mat["m"].reshape((c2,1), order='F')
x0 = np.zeros((c1*c1, 1))

ind = np.nonzero(Af)
indl = list(map(list, zip(ind[0], ind[1])))
vals = Af[ind].tolist()[0]

#indlT = list(map(list, zip(ind[1], ind[0])))
print(time.time() - t1)

1.2377290725708008


In [155]:
t1 = time.time()
Atf = tf.sparse.SparseTensor(indices = indl, values = vals, dense_shape = [c2, c1*c1])
#AT = tf.sparse.SparseTensor(indices = indlT, values = vals, dense_shape = [c1*c1, c2])
y = tf.convert_to_tensor(m, dtype='float32')
x = tf.Variable(tf.zeros((c1*c1, 1)))

yMax = tf.norm(y - tf.sparse.sparse_dense_matmul(Atf, x))
optimizer = tf.train.GradientDescentOptimizer(0.05)
train = optimizer.minimize(yMax)

init = tf.global_variables_initializer()
print(time.time() - t1)

3.51521897315979


In [156]:
t1 = time.time()

sess = tf.Session()

sess.run(init)
print(time.time() - t1)

t2 = time.time()
for i in range(1000):
    sess.run(train)
t3 = time.time()
elapsed = t3 - t2
print(sess.run(tf.norm(yMax)))
print(elapsed)

4.1358866691589355
20.226526
8.602668046951294


In [3]:
t1 = time.time()
mat = scipy.io.loadmat('/Users/aditya/LocalDocs/XRAYDATA/Walnut/Data82.mat')
norm = 0
mult = 1
c1 = 82*mult
c2 = 9840*mult

Af = mat["A"]/norms[norm] #norm
m = mat["m"].reshape((c2,1), order='F')
x0 = np.zeros((c1*c1, 1))

ind = np.nonzero(Af)
indl = list(map(list, zip(ind[0], ind[1])))
vals = Af[ind].tolist()[0]

#indlT = list(map(list, zip(ind[1], ind[0])))
print(time.time() - t1)

1.1924188137054443


In [4]:
def updateImage(A, f, g, it = 500, lR = 0.05, reg = 0):
    At = A.transpose()
    for i in range(it):
        t1 = time.time()
        yMax = np.subtract(g, A.dot(f))
        f = f + lR * At.dot(yMax)
        if i % 1000 == 0: print(np.sum(np.abs(yMax)))
    return f

In [5]:
x0 = np.zeros((c1*c1, 1))
a = time.time()
newF = updateImage(Af, np.asmatrix(x0), np.asmatrix(m), it = 1000)
print(time.time() - a)

3537.814775381415
2.1097049713134766


In [6]:
np.sum(np.abs(np.asmatrix(m) - Af.dot(newF)))

215.53340629372195

In [20]:
np.savetxt("./328Rec.csv", newF)

In [30]:
t1 = time.time()
mat = scipy.io.loadmat('/Users/aditya/LocalDocs/XRAYDATA/Walnut/Data328.mat')
norm = 2
mult = 4
c1 = 82*mult
c2 = 9840*mult

Af = mat["A"]/norms[norm] #norm
m = mat["m"].reshape((c2,1), order='F')
x0 = np.zeros((c1*c1, 1))

ind = np.nonzero(Af)
indl = list(map(list, zip(ind[0], ind[1])))
vals = Af[ind].tolist()[0]

#indlT = list(map(list, zip(ind[1], ind[0])))
print(time.time() - t1)

28.896011114120483


In [31]:
def icdUpdate(A, f, g, cols = c1, it=1):
    time1 = time.time()
    cols = cols*cols
    err = np.subtract(g, np.matmul(A, f))
    print(np.sum(np.abs(err)))
    for j in range(it):
        order = np.random.choice(cols, cols, replace=False)
        for i in order:
            t1 = np.sum(np.multiply(-1.0*err, A[:,i]))
            t2 = np.sum(np.multiply(A[:,i], A[:,i]))
            alpha = max(np.divide(-1*t1,t2), -f[i])
            f[i] += alpha
            err -= A[:,i]*alpha
        print(np.sum(np.abs(err)))
        print(time.time() - time1)
    return(f)

In [34]:
x0 = np.zeros((c1*c1, 1))
b = time.time()
newF = icdUpdate(Af.todense(), np.asmatrix(newF), np.asmatrix(m), it=10)
print(time.time() - b)

379.928111007222
375.05254040407596
117.90142607688904
370.9775497937253
184.40092515945435
367.49806325034183
243.99310994148254
364.14571000431397
299.8990581035614
361.3920482414469
353.4891359806061
358.9259304709896
411.43411803245544
356.53311784492263
468.2489261627197
354.3974025368966
526.5908288955688
352.4657174389237
583.7516441345215
350.69920105263657
644.25031208992
665.9008409976959


In [35]:
np.savetxt("../GPUreconstruct/icdRec328.csv", newF)

In [24]:
np.sum(np.abs(np.asmatrix(m) - Af.dot(newF)))

1658.5683026030804

In [34]:
def sgdUpdate(A, f, g, rows = c2, it=5, lR = 0.05):
    err = np.subtract(g, np.matmul(A, f))
    print(np.sum(np.abs(err)))
    for j in range(it):
        for i in range(rows):
            alpha = lR * A[i,:].transpose() * (g[i] - np.matmul(A[i,:],f))
            f += alpha
        if j % 100 == 0:
            print(np.sum(np.abs(np.subtract(g, np.matmul(A, f)))))
    return(f)

In [35]:
x0 = np.zeros((c1*c1, 1))
c = time.time()
newF = sgdUpdate(Af.todense(), np.asmatrix(x0), np.asmatrix(m), it=500)
print(time.time() - c)

3537.814775381415
3434.7055030834636


KeyboardInterrupt: 

In [162]:
t1 = time.time()
mat = scipy.io.loadmat('/Users/aditya/LocalDocs/XRAYDATA/Walnut/Data164.mat')
norm = 1
mult = 2
c1 = 82*mult
c2 = 9840*mult

Af = mat["A"]/norms[norm] #norm
m = mat["m"].reshape((c2,1), order='F')
x0 = np.zeros((c1*c1, 1))

ind = np.nonzero(Af)
indl = list(map(list, zip(ind[0], ind[1])))
vals = Af[ind].tolist()[0]

#indlT = list(map(list, zip(ind[1], ind[0])))
print(time.time() - t1)

5.168100833892822


In [10]:
a = time.time()
newx = icdUpdate(Af.todense(), np.asmatrix(x0), np.asmatrix(m), it=25)
b = time.time()
print(b-a)

98.6422948781917
98.58857952086946
98.55332757724346
98.4901677328936
98.44631541741711
98.420901418483
98.39483660462098
98.36169975379696
98.33505061911572
98.28942984527743
98.25839357305082
98.23709263174288
98.21484728566374
98.18558870369873
98.17685188786467
98.16686383191222
98.14212930564544
98.12528346090886
98.10159999541762
98.09525174233528
98.07589831683525
98.06724970136136
98.04810186397404
98.03963792670413
98.02704296641485
98.01385180804323
22.204486846923828


In [11]:
sgdX = sgdUpdate(Af.todense(), np.asmatrix(x0), np.asmatrix(m))

98.01385180804326
98.00906427709577
98.00432589705935
97.99964777252242
97.99503346386895
97.990469988012


In [119]:
np.savetxt('../recICDSGDcomb82.csv', x0)

In [181]:
with tf.Session() as sess:
    print(sess.run(tf.tensordot(tf.ones((4, 1)), tf.zeros((4, 1)), axes = 0)))

[[[[0.]
   [0.]
   [0.]
   [0.]]]


 [[[0.]
   [0.]
   [0.]
   [0.]]]


 [[[0.]
   [0.]
   [0.]
   [0.]]]


 [[[0.]
   [0.]
   [0.]
   [0.]]]]


In [185]:
t1 = time.time()
mat = scipy.io.loadmat('/Users/aditya/LocalDocs/XRAYDATA/Walnut/Data82.mat')
norm = 0
mult = 1
c1 = 82*mult
c2 = 9840*mult

Af = mat["A"]/norms[norm] #norm
m = mat["m"].reshape((c2,1), order='F')
x0 = np.zeros((c1*c1, 1))

ind = np.nonzero(Af)
indl = list(map(list, zip(ind[0], ind[1])))
vals = Af[ind].tolist()[0]

#indlT = list(map(list, zip(ind[1], ind[0])))
print(time.time() - t1)

3.502419948577881


In [212]:
Atf = tf.sparse.to_dense(tf.sparse.SparseTensor(indices = indl, values = vals, dense_shape = [9840, 82*82]))
y = tf.convert_to_tensor(m, dtype='float32')
x = tf.Variable(tf.zeros((82*82, 1)))
err = tf.Variable(tf.zeros((9840, 1)))
rangeVar = tf.convert_to_tensor(np.arange(6724))

indx = tf.placeholder('int32', shape=None)

Ai = tf.slice(Atf, [0, indx], [9840, 1])
xi = x[indx]

firstErr = err.assign(y - tf.matmul(Atf, x))
t1 = tf.reduce_sum(tf.multiply(-1*err, Ai))
t2 = tf.reduce_sum(tf.multiply(Ai, Ai))
alpha = tf.math.maximum(-1*t1/t2, -1*xi)
newErr = err.assign(err - Ai*alpha)
updatex = x.assign(x + tf.transpose(tf.multiply(tf.one_hot([indx], 82*82), alpha)))


In [223]:
rangeVar = tf.convert_to_tensor(np.arange(5))
a = time.time()
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
sess.run(firstErr)
for j in range(1):
    #tmpVar = sess.run(rangeVar)
    for i in sess.run(rangeVar):
        sess.run(newErr, feed_dict={indx: i})
        sess.run(updatex, feed_dict={indx: i})
        b = time.time()
        print(b-a)

35.63717198371887
35.88363289833069
36.147653102874756
36.39313888549805
36.640413999557495


In [128]:
def getErr(y, Atf, x, err):
    return y-tf.matmul(Atf, x)
    
def body(i, A = Atf, x = x, err = err):
    Ai = tf.slice(A, [0, i], [9840, 1])
    xi = x[i]
    t1 = tf.reduce_sum(tf.multiply(-1*err, Ai))
    t2 = tf.reduce_sum(tf.multiply(Ai, Ai))
    alpha = tf.math.maximum(-1*t1/t2, -1*xi)
    return xi + alpha
    
def cond(i, A, x, err, iters):
    return tf.less(i, iters)



res = tf.map_fn(cond, body, [0, Atf, tf.range(6724), err, 6724])

TypeError: body() takes from 1 to 4 positional arguments but 5 were given

In [152]:
with tf.Session() as sess:
    print("hi")
    print(tf.random.shuffle(tf.convert_to_tensor(np.arange(6724))).eval())


hi
[  31 1157  451 ... 5980 1709  168]


In [ ]:
mult = 2
c1 = 9840 * mult
c2 = 82 * mult

dtype = tf.float32

matrix2 = tf.convert_to_tensor(np.ones((c1, c2*c2)), dtype=dtype)
mat3 = tf.transpose(matrix2)
x = tf.Variable(tf.ones((c2*c2, 1), dtype=dtype))
y = tf.convert_to_tensor(np.ones((c1, 1)), dtype=dtype)

product = tf.subtract(y, tf.matmul(matrix2, x))
product2 = x.assign(x + tf.scalar_mul(0.05, tf.matmul(mat3, product)))

sess = tf.Session()

sess.run(tf.global_variables_initializer())
iters = 10

sess.run(product.op)

start = time.time()
for i in range(iters):
    sess.run(product.op)
end = time.time()
#ops = n**3 + (n-1)*n**2 # n^2*(n-1) additions, n^3 multiplications
elapsed = (end - start)
#rate = iters*ops/elapsed/10**9
print('\n matmul took: %.2f sec' % (elapsed/iters))

In [30]:
import scipy.io
import numpy as np
import tensorflow as tf
import time
import scipy.sparse
norms = [66.71730, 33.34396, 16.92447, 372.3057, 186.1538]

mult = 1
norm = 3
matname = '/Users/aditya/LocalDocs/XRAYDATA/LotusXRAY/LotusData128.mat'

print(it)
print(mult)
print("starting!")
t1 = time.time()

mat = scipy.io.loadmat(matname)
norm = 0
c1 = 128 * mult
c2 = 51480

Af = mat["A"]/norms[norm]  #norm
m = mat["m"].reshape((c2,1), order='F')
x0 = np.zeros((c1*c1, 1))

ind = np.nonzero(Af)
indl = list(map(list, zip(ind[0], ind[1])))
vals = Af[ind].tolist()[0]

#indlT = list(map(list, zip(ind[1], ind[0])))
print('\nprecompute took: %.4f sec\n\n' % (time.time() - t1))
t1 = time.time()

Atf = tf.sparse.SparseTensor(indices = indl, values = vals, dense_shape = [c2, c1*c1])
#AT = tf.sparse.SparseTensor(indices = indlT, values = vals, dense_shape = [c1*c1, c2])
y = tf.convert_to_tensor(m, dtype='float32')
x = tf.Variable(tf.zeros((c1*c1, 1)))

yMax = tf.norm(y - tf.sparse.sparse_dense_matmul(Atf, x), 2)
optimizer = tf.train.GradientDescentOptimizer(0.05)
train = optimizer.minimize(yMax)
#update = x.assign(x + tf.scalar_mul(0.05, tf.sparse.sparse_dense_matmul(AT, yMax)))
init = tf.global_variables_initializer()

print('\nTensorflow load took: %f sec\n\n' % (time.time() - t1))
t1 = time.time()

sess = tf.Session()

sess.run(init)
sess.run(train)

print('\nTensorflow initialized: %f sec\n\n' % (time.time() - t1))


t2 = time.time()

for i in range(1000):
    sess.run(train)

t3 = time.time()
elapsed = t3 - t2

print('\n1k training took: %f sec\n\n' % elapsed)
print('\nL2-norm: %f\n\n' % sess.run(yMax))
print('\nL1-norm: %f\n\n' % sess.run(tf.norm(y - tf.sparse.sparse_dense_matmul(Atf, x), 1 )))


t2 = time.time()

for i in range(4000):
    sess.run(train)

t3 = time.time()
elapsed = t3 - t2

print('\n5k training took: %f sec\n\n' % elapsed)
print('\nL2-norm: %f\n\n' % sess.run(yMax))
print('\nL1-norm: %f\n\n' % sess.run(tf.norm(y - tf.sparse.sparse_dense_matmul(Atf, x), 1 )))


t2 = time.time()

for i in range(5000):
    sess.run(train)

t3 = time.time()
elapsed = t3 - t2

print('\n10k training took: %f sec\n\n' % elapsed)
print('\nL2-norm: %f\n\n' % sess.run(yMax))
print('\nL1-norm: %f\n\n' % sess.run(tf.norm(y - tf.sparse.sparse_dense_matmul(Atf, x), 1 )))


t2 = time.time()

for i in range(5000):
    sess.run(train)

t3 = time.time()
elapsed = t3 - t2

print('\n15k training took: %f sec\n\n' % elapsed)
print('\nL2-norm: %f\n\n' % sess.run(yMax))
print('\nL1-norm: %f\n\n' % sess.run(tf.norm(y - tf.sparse.sparse_dense_matmul(Atf, x), 1 )))

10000
1
starting!

precompute took: 58.0050 sec



Tensorflow load took: 48.539661 sec



Tensorflow initialized: 72.785091 sec



1k training took: 65.098375 sec



L2-norm: 10.981655



L1-norm: 1695.640015



5k training took: 248.391602 sec



L2-norm: 8.555499



L1-norm: 1386.659912



10k training took: 302.229806 sec



L2-norm: 8.496877



L1-norm: 1382.160889



15k training took: 302.026403 sec



L2-norm: 8.474133



L1-norm: 1380.409912




In [32]:
import scipy.io
import numpy as np
import tensorflow as tf
import time
import scipy.sparse
norms = [66.71730, 33.34396, 16.92447, 372.3057, 186.1538]

mult = 2
norm = 4
matname = '/Users/aditya/LocalDocs/XRAYDATA/LotusXRAY/LotusData256.mat'

print(it)
print(mult)
print("starting!")
t1 = time.time()

mat = scipy.io.loadmat(matname)
norm = 0
c1 = 128 * mult
c2 = 51480

Af = mat["A"]/norms[norm]  #norm
m = mat["m"].reshape((c2,1), order='F')
x0 = np.zeros((c1*c1, 1))

ind = np.nonzero(Af)
indl = list(map(list, zip(ind[0], ind[1])))
vals = Af[ind].tolist()[0]

#indlT = list(map(list, zip(ind[1], ind[0])))
print('\nprecompute took: %.4f sec\n\n' % (time.time() - t1))
t1 = time.time()

Atf = tf.sparse.SparseTensor(indices = indl, values = vals, dense_shape = [c2, c1*c1])
#AT = tf.sparse.SparseTensor(indices = indlT, values = vals, dense_shape = [c1*c1, c2])
y = tf.convert_to_tensor(m, dtype='float32')
x = tf.Variable(tf.zeros((c1*c1, 1)))

yMax = tf.norm(y - tf.sparse.sparse_dense_matmul(Atf, x), 2)
optimizer = tf.train.GradientDescentOptimizer(0.05)
train = optimizer.minimize(yMax)
#update = x.assign(x + tf.scalar_mul(0.05, tf.sparse.sparse_dense_matmul(AT, yMax)))
init = tf.global_variables_initializer()

print('\nTensorflow load took: %f sec\n\n' % (time.time() - t1))
t1 = time.time()

sess = tf.Session()

sess.run(init)
sess.run(train)

print('\nTensorflow initialized: %f sec\n\n' % (time.time() - t1))


t2 = time.time()

for i in range(1000):
    sess.run(train)

t3 = time.time()
elapsed = t3 - t2

print('\n1k training took: %f sec\n\n' % elapsed)
print('\nL2-norm: %f\n\n' % sess.run(yMax))
print('\nL1-norm: %f\n\n' % sess.run(tf.norm(y - tf.sparse.sparse_dense_matmul(Atf, x), 1 )))


t2 = time.time()

for i in range(4000):
    sess.run(train)

t3 = time.time()
elapsed = t3 - t2

print('\n5k training took: %f sec\n\n' % elapsed)
print('\nL2-norm: %f\n\n' % sess.run(yMax))
print('\nL1-norm: %f\n\n' % sess.run(tf.norm(y - tf.sparse.sparse_dense_matmul(Atf, x), 1 )))


t2 = time.time()

for i in range(5000):
    sess.run(train)

t3 = time.time()
elapsed = t3 - t2

print('\n10k training took: %f sec\n\n' % elapsed)
print('\nL2-norm: %f\n\n' % sess.run(yMax))
print('\nL1-norm: %f\n\n' % sess.run(tf.norm(y - tf.sparse.sparse_dense_matmul(Atf, x), 1 )))



10000
2
starting!

precompute took: 172.2700 sec



Tensorflow load took: 251.123501 sec



Tensorflow initialized: 148.660745 sec



1k training took: 190.410744 sec



L2-norm: 74.349327



L1-norm: 12067.123047



5k training took: 778.634502 sec



L2-norm: 9.307591



L1-norm: 1474.362061



10k training took: 970.082985 sec



L2-norm: 8.505434



L1-norm: 1377.253784


